In [20]:
import hnswlib
from torch import nn
import torch
import argparse
import numpy as np
import pandas as pd
import scanpy as sc
import os
import anndata
import math

from modules import network,mlp
from utils import yaml_config_hook,save_model

In [21]:
parser = argparse.ArgumentParser()
config = yaml_config_hook("config/config.yaml")
for k, v in config.items():
    parser.add_argument(f"--{k}", default=v, type=type(v))
args = parser.parse_args([])
if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

# torch.manual_seed(args.seed)
# torch.cuda.manual_seed_all(args.seed)
# torch.cuda.manual_seed(args.seed)
# np.random.seed(args.seed)
class_num = args.classnum

In [22]:
sparse_X = scipy.sparse.load_npz('data/filtered_Counts.npz')
annoData = pd.read_table('data/annoData.txt')
y = annoData["cellIden"].to_numpy()
high_var_gene = 3000
# normlization and feature selection
adataSC = anndata.AnnData(X=sparse_X, obs=np.arange(sparse_X.shape[0]), var=np.arange(sparse_X.shape[1]))
sc.pp.filter_genes(adataSC, min_cells=10)
adataSC.raw = adataSC
sc.pp.highly_variable_genes(adataSC, n_top_genes=high_var_gene, flavor='seurat_v3')
sc.pp.normalize_total(adataSC, target_sum=1e4)
sc.pp.log1p(adataSC)

adataNorm = adataSC[:, adataSC.var.highly_variable]
dataframe = adataNorm.to_df()
x_ndarray = dataframe.values.squeeze()
y_ndarray = np.expand_dims(y, axis=1)
print(x_ndarray.shape,y_ndarray.shape)
dataframe.head()

C:\Users\Tom-G\.conda\envs\torchenv\lib\site-packages\anndata\_core\anndata.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if index_name in anno:


(8569, 3000) (8569, 1)


,2,4,10,13,33,41,45,47,62,68,...,19854,19871,19883,20021,20040,20073,20108,20109,20121,20124
0,0.0,0.0,1.302199,0.0,0.0,0.0,0.0,0.0,0.000000,0.36896,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.36896,0.0,0.637877,0.368960
1,0.0,0.0,1.351171,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.888292,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.711146,0.00000,...,0.0,0.000000,0.93785,0.0,0.0,0.0,0.00000,0.0,0.937850,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.509045,0.00000,...,0.0,0.509045,0.00000,0.0,0.0,0.0,0.00000,0.0,0.509045,0.509045


In [23]:
from torch.utils.data import DataLoader,TensorDataset
scDataset = TensorDataset(torch.tensor(x_ndarray, dtype=torch.float32),
                              torch.tensor(y_ndarray, dtype=torch.float32))

# scTrainLength = int(len(scDataset) * 0.8)
# scValidLength = len(scDataset) - scTrainLength
# scTrain, scValid = random_split(scDataset, [scTrainLength, scValidLength])

scDataLoader = DataLoader(scDataset, shuffle=True, batch_size=len(scDataset),drop_last=True)
# scValidDataLoader = DataLoader(scValid, shuffle=True, batch_size=256,drop_last=True)

for features, labels in scDataLoader:
    print(len(features[-1]))
    print(len(features))
    print(len(labels))
    break

3000
8569
8569


In [24]:
num_elements=len(scDataset)
dim=high_var_gene
print(num_elements,dim)
p=hnswlib.Index(space='l2',dim=dim)
p.init_index(max_elements=num_elements, ef_construction=100, M=16)
p.set_ef(100)
p.set_num_threads(4)
p.add_items(x_ndarray)

8569 3000


In [25]:
mlp = mlp.MLP(num_genes=args.num_genes)
model = network.Network(mlp, args.feature_dim, args.classnum)
model = model.to('cuda')
# optimizer / loss
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

In [28]:
for features,psedulabels in scDataLoader:
    

    labels, distances = p.knn_query(x_ndarray[0], k=64)
    # print(labels,distances)
    print(distances.shape)
    print(distances[0][-1])
    print(type(labels))
    labels=labels.squeeze()
    # print(labels)

    contrastive=features[labels.tolist()]
    print(contrastive.shape)


    input=np.row_stack((contrastive,x_ndarray[0]))
    # print(input.shape)
    input_1=input.copy()
    np.random.shuffle(input)

    input=torch.from_numpy(input).to('cuda')
    input_1=torch.from_numpy(input_1).to('cuda')

    print(model(input))
    print(model(input_1))
    print(input.shape,input_1.shape)
    
    break

(1, 64)
396.69873
<class 'numpy.ndarray'>
torch.Size([64, 3000])


TypeError: forward() missing 1 required positional argument: 'x_j'